FUNCTION
===================

TIME AND DATE
===================

INPUT(): "reread" a variable according to a new informat, char to numeric (IN for INteger)  
take the first argument and "reads" it as if it were coming from a file, according to the INFORMAT listed as the second argument.  

PUT: use formatted value of a variable to create new variable, numeric to char 
it "writes out" the value of the first argument, according to the FORMAT in second argument. By "write out" we actually mean assign the value to the variable on the left of the equal sign.  

STRING
=========
concantenate: `||`  

 * CAT does not remove leading or trailing blanks, and does not  
   insert separators.  

 * CATT trims trailing blanks, but not leading blanks, and does not  
   insert separators.  

 * CATS strips both leading and trailing blanks, and does not  
   insert separators.  

 * CATX strips both leading and trailing blanks, and inserts  
   separators. The first argument to CATX specifies the separator.  

In [1]:

DATA EX_10; 
   INPUT LONG_STR $ 1-80;
   ARRAY PIECES[5] $ 10   
      PIECE1-PIECE5;
   DO I = 1 TO 5;
      PIECES[I] = SCAN(LONG_STR,I,',;.! ');
   END;
   DROP LONG_STR I;
DATALINES4;
THIS LINE,CONTAINS!FIVE.WORDS
ABCDEFGHIJKL XXX;YYY
;;;;
PROC PRINT DATA=EX_10 NOOBS;
   TITLE 'Listing of Example 10';
RUN;

PIECE1,PIECE2,PIECE3,PIECE4,PIECE5
THIS,LINE,CONTAINS,FIVE,WORDS
ABCDEFGHIJ,XXX,YYY,,


In [ ]:

DATA EX_12;
   LENGTH A B C D E $ 1;
   INPUT A B C D E X Y;
DATALINES;
M f P p D 1 2
m f m F M 3 4
;
DATA UPPER;
   SET EX_12;
   ARRAY ALL_C[*] _CHARACTER_;
   DO I = 1 TO DIM(ALL_C);
      ALL_C[I] = UPCASE(ALL_C[I]);
   END;
   DROP I;
RUN;

PROC PRINT DATA=UPPER NOOBS;
   TITLE 'Listing of UPPER';
RUN;